# Imports

In [2]:
# <your imports>
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, XLMRobertaXLConfig
from transformers import EarlyStoppingCallback

import sys
sys.path.append('../src')
from preprocessing import Preprocessing
from matplotlib import pyplot as plt

# models
from models import BertClassifier

# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
with open("metric_test.txt", 'w'):
    pass

In [3]:
def custom_f1(p):
    pred, labels = p
    with open("metric_test.txt", 'a') as f:
        f.write(" ".join(map(str, labels)))
        f.write("\n")
        for pred_i in pred:
            f.write(" ".join(map(str, pred_i)))
            f.write("\n")
    pred = np.argmax(pred, axis=1)

    f1_scores = f1_score(y_true=labels, y_pred=pred, average=None)
    final_score = np.mean(f1_scores[1:])
    # return {"custom F1 score": final_score}
    return {
        "F1 for class 0": f1_scores[0],
        "F1 for class 1": f1_scores[1],
        "F1 for class 2": f1_scores[2]
    }

def compute_metrics(p):
    pred, labels = p
    # print(pred, labels)
    # print(np.unique(labels, return_counts=True), labels.shape)
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Data loading/preprocessing

In [4]:
preprocessor = Preprocessing()

# result = preprocessor.get_dataloaders()
result = preprocessor.get_datasets()

train_set = result["train_set"]
val_set = result["val_set"]
# plt.hist(val_set.labels, bins=3)

# Model initialization

In [5]:
conf = OmegaConf.load("../config.yaml").general
for i in conf.keys():
    print(f'{i}: {conf[i]}')

dataset_name: ruSentNE
batch_size: 64
device: cpu:0
dataloader_shuffle: True
pretrained_model: bert-base-uncased
num_classes: 3
num_workers: 1
lr: 1e-06
epochs: 5
eps: 1e-08
optimizer: AdamW
eval_steps: 10
logging_steps: 10


In [46]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment", 
    # 'cardiffnlp/twitter-xlm-roberta-base-sentiment',
    num_labels=3
)

In [47]:
# for param in model.roberta.parameters():
#     param.requires_grad = False

In [48]:
# model.classifier = nn.Linear(in_features=768, out_features=3, bias=True)

In [49]:
# help(AutoModelForSequenceClassification)

In [50]:
# # model = BertForSequenceClassification.from_pretrained(conf.pretrained_model,
# #                                                       num_labels=conf.num_classes,
# #                                                       output_attentions=False,
# #                                                       output_hidden_states=False)

# model = XLMRobertaXLModel.from_pretrained(conf.pretrained_model,
#                                            num_labels=conf.num_classes,
#                                            output_attentions=False,
#                                            output_hidden_states=False)

In [51]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=conf.eval_steps,
    per_device_train_batch_size=conf.batch_size,
    per_device_eval_batch_size=conf.batch_size,
    num_train_epochs=conf.epochs,
    seed=42,
    logging_steps=conf.logging_steps,
    # learning_rate=conf.lr,
    learning_rate=5e-7,
    load_best_model_at_end=True,
)

In [52]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    # compute_metrics=compute_metrics,
    compute_metrics=custom_f1,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [53]:
# help(Trainer)

## Train loop

In [54]:
# API key for wandb: 1fef7fd74ee7f9dfeb33b86d13b60f8ffe9c968a

In [55]:
trainer.train()

Step,Training Loss,Validation Loss,F1 for class 0,F1 for class 1,F1 for class 2
10,0.982900,0.898993,0.000000,0.836553,0.000000
20,0.944300,0.896971,0.000000,0.836553,0.000000
30,0.906200,0.894953,0.000000,0.836553,0.000000


KeyboardInterrupt: 

# Custom Trainer

In [ ]:
from trainer import CustomTrainer

In [ ]:
preprocessor = Preprocessing()

result = preprocessor.get_dataloaders()

train_dl = result["train_dl"]
val_dl = result["val_dl"]

In [ ]:
trainer = CustomTrainer(model.cuda())

In [ ]:
trainer = CustomTrainer(model.cuda())

In [ ]:
trainer.train(train_dl, val_dl)

# Experiments

In [36]:
from sklearn.metrics import f1_score, classification_report
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]

f1_score(y_true, y_pred, average=None)

array([0.8, 0. , 0. ])

In [36]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         2

    accuracy                           0.33         6
   macro avg       0.22      0.33      0.27         6
weighted avg       0.22      0.33      0.27         6



In [38]:
" ".join(map(str, y_true))

'0 1 2 0 1 2'